## Rationale
Show, on the surface, where this full-darkness experiments produces time-structured signals.  <br/>
We will use model-free analysis (simple correlation between tasks) that shows us where we have explainable signals,  <br/>
these are the signals we will need to start explaining in the next step.  <br/>

## Analsyis steps

### GazeCW/GazeCCW task

- [x] Load GazeCW and GazeCCW data (_see [cormaps.py](cormaps.py)_)
- [x] Mask all data with brain mask (_see [cormaps.py](cormaps.py)_)
- [x] Deal with fold permutations (_see [cormaps.py](cormaps.py)_)
- [x] Do all correlation (_see [cormaps.py](cormaps.py)_)
- [x] Average correlations maps (_see [cormaps.py](cormaps.py)_)
- [x] Draw correlation maps on pycortex flatmap (_see below_)
- [x] Create static webgl (_see below_)
- [x] upload to webapp server

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nb
import shutil
import warnings
import re
import cortex
import time
import glob
import matplotlib.pyplot as plt
from pycortex_utils import draw_cortex
from pathlib import Path
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001','sub-002']
tasks = ['GazeCWGazeCCW', 
         'GazeColumnsGazeLines']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_exp'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)

# Plot parameters
cmap_cor = 'RdBu_r'
save_svg = False
xfm_name = 'identity2.fmriprep'
alpha_range = [0, 1]
cor_range = [-1.0, 1.0]
recache = False
inflated_maps_names = ['GazeCWGazeCCW_cor','GazeColumnsGazeLines_cor']

In [ ]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

# Add transform to pycortex db
for subject in subjects: 
    print('add transform')
    file_list = sorted(glob.glob("{base_dir}/derivatives/pp_data/{sub}/cor/*.nii.gz".format(base_dir = base_dir, sub = subject)))
    ref_file = file_list[0]
    transform = cortex.xfm.Transform(np.identity(4), ref_file)
    transform.save(subject, xfm_name, 'magnet')

In [ ]:
for subject in subjects: 
    maps_names = []
    volumes = {}
    exec('{}_volumes = {{}}'.format(re.sub('-','_',subject)))
    for task in tasks:
        # Define and create folder
        cor_dir = '{}/{}/cor'.format(pp_dir, subject)
        flatmaps_dir = '{}/{}/cor/flatmaps'.format(pp_dir, subject)
        try: os.makedirs(flatmaps_dir) 
        except: pass

        # Load parameters
        cor_im = nb.load('{}/{}_{}_timeseries_cor.nii.gz'.format(cor_dir,subject,task))
        cor_mat = cor_im.get_fdata()

        # Refit reference frame index
        cor_param = {'data': cor_mat, 'cmap': cmap_cor, 'vmin': cor_range[0], 'vmax': cor_range[1], 'cortex_type': 'Volume',
                     'cbar': 'discrete', 'cbar_label': 'Correlation (a.u.)', 'description': 'Timeseries correlation: {}'.format(task), 
                     'curv_brightness': 1, 'curv_contrast': 0.3, 'add_roi': False, 'subject': subject, 'xfmname': xfm_name}
        exec('{}_cor_param = cor_param'.format(task))
        maps_names.append('{}_cor'.format(task))
            
    # Flatmaps
    for maps_name in maps_names:
        exec('{}_volume, fig = draw_cortex(**{}_param)'.format(maps_name,maps_name))
        print("{}_task-{}".format(subject, maps_name))
        exec("plt.savefig('{}/{}_task-{}.pdf')".format(flatmaps_dir, subject, maps_name))
        plt.show()
        exec('vol_description = {}_param["description"]'.format(maps_name))
        exec('volume = {}_volume'.format(maps_name))
        volumes.update({vol_description:volume})
    
    # Save Webgl
    webgl_dir = '{}/derivatives/webgl_data/{}/'.format(base_dir, subject)
    if os.path.exists(webgl_dir):
        print('deleting old : {}'.format(webgl_dir))
        shutil.rmtree(webgl_dir)
    
    print('saving: {}'.format(webgl_dir))
    try: os.makedirs(webgl_dir)
    except: pass
    cortex.webgl.make_static(outpath=webgl_dir, data=volumes, recache=recache, title=subject)
